In [72]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
from torch import distributions as dist

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [30]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']
X.shape, y.shape, set(y)

((150, 4), (150,), {0, 1, 2})

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [45]:
ds_train = TensorDataset(
    torch.Tensor(X_train), 
    torch.Tensor(y_train).long()
)

ds_test = TensorDataset(
    torch.Tensor(X_test), 
    torch.Tensor(y_test).long()
)

batch_size=16
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_test = DataLoader(ds_test, batch_size, shuffle=True)

In [69]:
model = nn.Sequential(
    nn.Linear(4, 10),
#     nn.ReLU(),
    nn.Linear(10, 20),
# #     nn.ReLU(),
    nn.Linear(20, 3)
)

opt = optim.Adam(model.parameters(), 4e-3)
criterion = nn.CrossEntropyLoss()
number_of_epochs=40
model

Sequential(
  (0): Linear(in_features=4, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=20, bias=True)
  (2): Linear(in_features=20, out_features=3, bias=True)
)

In [70]:
for i in range(number_of_epochs):
    train_loss = 0
    model = model.train()
    y_train_predicted = []
    y_train_true = []
    for iteration, sample in enumerate(dl_train):
        imgs, y = sample

        model.zero_grad()
        output = model(imgs)
        loss = criterion(output, y)
        train_loss += loss.item() / imgs.size(0)
        loss.backward()
        opt.step()
        _, pred = torch.max(output, dim=1)
        y_train_predicted.extend(pred.detach().cpu().tolist())
        y_train_true.extend(y.detach().cpu().tolist())
        
    val_loss = 0
    y_predicted = []
    y_true = []

    with torch.no_grad():
        model = model.eval()
        for iteration, sample in enumerate(dl_test):
            imgs, y = sample

            output = model(imgs)
            loss = criterion(output, y)
            val_loss += loss.item() / imgs.size(0)
            _, pred = torch.max(output, dim=1)
            y_predicted.extend(pred.cpu().tolist())
            y_true.extend(y.cpu().tolist())
            
    val_acc = accuracy_score(y_true, y_predicted)
    print('#Epoch: {}, train loss: {}, test loss: {}, val_acc: {}'.format(i, train_loss, val_loss, val_acc))
    

#Epoch: 0, train loss: 0.6946636997163296, test loss: 0.7056384943425655, val_acc: 0.64
#Epoch: 1, train loss: 0.6657244525849819, test loss: 0.5773552842438221, val_acc: 0.3
#Epoch: 2, train loss: 0.6065041124820709, test loss: 0.3926589712500572, val_acc: 0.7
#Epoch: 3, train loss: 0.43259335309267044, test loss: 0.40072745084762573, val_acc: 0.7
#Epoch: 4, train loss: 0.41520318388938904, test loss: 0.3077712208032608, val_acc: 0.96
#Epoch: 5, train loss: 0.34135354682803154, test loss: 0.433471018448472, val_acc: 0.82
#Epoch: 6, train loss: 0.2684185318648815, test loss: 0.29664959385991096, val_acc: 0.88
#Epoch: 7, train loss: 0.27914427779614925, test loss: 0.23491299338638783, val_acc: 0.88
#Epoch: 8, train loss: 0.22679435648024082, test loss: 0.3587693180888891, val_acc: 0.98
#Epoch: 9, train loss: 0.20056397281587124, test loss: 0.07554511539638042, val_acc: 0.98
#Epoch: 10, train loss: 0.2070946916937828, test loss: 0.2697750264778733, val_acc: 1.0
#Epoch: 11, train loss: 0.

# Gaussian estimator

In [87]:
true_d = dist.multivariate_normal.MultivariateNormal(torch.Tensor([1,1]), torch.Tensor([[1,0],[0,1]]))
X = true_d.sample((100,))
des_loss = - true_d.log_prob(X)

print("desired loss:", des_loss.mean())
mean_v = torch.autograd.Variable(torch.Tensor(np.random.rand(2)), requires_grad=True)
cov_v = torch.autograd.Variable(torch.Tensor(np.diag(np.abs(np.random.rand(2)))), requires_grad=True)

params = [mean_v, cov_v]


opt = optim.Adam(params, 0.01)

for i in range(100):
    opt.zero_grad()
    d = dist.multivariate_normal.MultivariateNormal(mean_v, cov_v)

    loss = - d.log_prob(X).mean()
    loss.backward()
    opt.step()
    for p in params:
        p.grad.data.zero_()

print("trained loss:", loss)

desired loss: tensor(2.7378)
trained loss: tensor(2.7533, grad_fn=<NegBackward>)


In [101]:
P = torch.rand(10)

P_var = torch.autograd.Variable(P, requires_grad=True)
print("pre", P_var)
opt = optim.Adam([P_var], 0.01)

for i in range(100):
    opt.zero_grad()
    loss = - torch.log(
        torch.nn.functional.softmax(P_var)
    ).sum()
    print(loss)
    loss.backward()
    opt.step()
    P_var.grad.data.zero_()

print("post", P_var)

pre tensor([0.6962, 0.2590, 0.8861, 0.3150, 0.0434, 0.5062, 0.8557, 0.8400, 0.7429,
        0.8529], requires_grad=True)
tensor(23.4042, grad_fn=<NegBackward>)
tensor(23.3813, grad_fn=<NegBackward>)
tensor(23.3594, grad_fn=<NegBackward>)
tensor(23.3382, grad_fn=<NegBackward>)
tensor(23.3180, grad_fn=<NegBackward>)
tensor(23.2987, grad_fn=<NegBackward>)
tensor(23.2802, grad_fn=<NegBackward>)
tensor(23.2627, grad_fn=<NegBackward>)
tensor(23.2460, grad_fn=<NegBackward>)
tensor(23.2303, grad_fn=<NegBackward>)
tensor(23.2154, grad_fn=<NegBackward>)
tensor(23.2013, grad_fn=<NegBackward>)
tensor(23.1881, grad_fn=<NegBackward>)
tensor(23.1757, grad_fn=<NegBackward>)
tensor(23.1639, grad_fn=<NegBackward>)
tensor(23.1529, grad_fn=<NegBackward>)
tensor(23.1426, grad_fn=<NegBackward>)
tensor(23.1328, grad_fn=<NegBackward>)
tensor(23.1237, grad_fn=<NegBackward>)
tensor(23.1152, grad_fn=<NegBackward>)
tensor(23.1073, grad_fn=<NegBackward>)
tensor(23.0999, grad_fn=<NegBackward>)
tensor(23.0930, grad_

/home/mprzewie/.anaconda3/envs/uj/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


# MNIST

In [8]:
transforms = tr.Compose([])

In [13]:
ds = MNIST('./data', train=True, target_transform=None, download=True, transform=transforms)
ds_test = MNIST('./data', train=False, target_transform=None, download=True, transform=transforms)

batch_size = 32
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

In [16]:
i, l = ds[0]
i